In [228]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from functools import reduce
from recommenders.amazon_recommender import AmazonRecommender
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the dataset for recommenders

In [229]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

### Approach 1: Probability of feature occurance in user reservations - recommended to use, final results used this approach

In [172]:
def prepare_users_df(interactions_df):

    # extract features as one-hot representation | base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']
    extracted_features = []
    
    # create data frames of features
    for feature in base_item_features:
        extracted_features.append(pd.get_dummies(interactions_df[feature], prefix = "u_" + feature))

    # merge all features into one data frame with user_id column
    users_df = reduce(lambda left, right: pd.merge(left, right, how = 'left', left_index = True, right_index = True), extracted_features)
    users_df.insert(0, 'user_id', interactions_df['user_id'])

    # in case of any NaN 
    users_df = users_df.fillna(0)
    
    # count probability by using one-hot encoded features
    agg_users_df = users_df.groupby(by = ['user_id']).sum()
    occurances = users_df['user_id'].value_counts().sort_index().tolist()
    agg_users_df.insert(0, 'occurances', occurances)
    agg_users_df = agg_users_df.iloc[:, 1:].div(agg_users_df['occurances'], axis = 0).reset_index(drop = True)
    agg_users_df.insert(0, 'user_id', interactions_df['user_id'].unique())
    
    # extract user features names
    user_features = list(users_df.columns.values[1:])

    return agg_users_df, user_features

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['u_term_WinterVacation', 'u_term_Easter', 'u_term_OffSeason', 'u_term_HighSeason', 'u_term_LowSeason', 'u_term_MayLongWeekend', 'u_term_NewYear', 'u_term_Christmas', 'u_length_of_stay_bucket_[0-1]', 'u_length_of_stay_bucket_[2-3]', 'u_length_of_stay_bucket_[4-7]', 'u_length_of_stay_bucket_[8-inf]', 'u_rate_plan_Standard', 'u_rate_plan_Nonref', 'u_room_segment_[0-160]', 'u_room_segment_[160-260]', 'u_room_segment_[260-360]', 'u_room_segment_[360-500]', 'u_room_segment_[500-900]', 'u_n_people_bucket_[1-1]', 'u_n_people_bucket_[2-2]', 'u_n_people_bucket_[3-4]', 'u_n_people_bucket_[5-inf]', 'u_weekend_stay_True', 'u_weekend_stay_False']


,user_id,u_term_WinterVacation,u_term_Easter,u_term_OffSeason,u_term_HighSeason,u_term_LowSeason,u_term_MayLongWeekend,u_term_NewYear,u_term_Christmas,u_length_of_stay_bucket_[0-1],...,u_room_segment_[160-260],u_room_segment_[260-360],u_room_segment_[360-500],u_room_segment_[500-900],u_n_people_bucket_[1-1],u_n_people_bucket_[2-2],u_n_people_bucket_[3-4],u_n_people_bucket_[5-inf],u_weekend_stay_True,u_weekend_stay_False
0,1,0.090909,0.0,0.681818,0.090909,0.136364,0.0,0.0,0.0,0.0,...,0.863636,0.136364,0.0,0.0,0.0,0.727273,0.181818,0.090909,0.772727,0.227273
40,50,0.043478,0.0,0.434783,0.304348,0.217391,0.0,0.0,0.0,0.0,...,0.565217,0.434783,0.0,0.0,0.0,0.173913,0.521739,0.304348,0.782609,0.217391
321,706,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000
547,115,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,1.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.000000,1.000000,0.000000
1270,1736,0.000000,0.0,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,1.000000
2785,96,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,1.0,...,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000,0.000000,0.000000,1.000000
11527,7779,1.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,1.000000,0.000000,0.000000,0.000000,1.000000


### Approach 2: Vector of most common features in user reservations - only for testing another approach, better to use previous one

In [292]:
def prepare_users_df(interactions_df):

    # get unique user ids
    user_ids = interactions_df['user_id'].unique()
    # extract most common values per base item features in all user reservations
    interactions_df = interactions_df.groupby(by = ['user_id']).agg(lambda x: x.value_counts().index[0]).reset_index(drop = True)

    # extract features as one-hot representation | base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']
    extracted_features = []
    
    # create data frames of features
    for feature in base_item_features:
        extracted_features.append(pd.get_dummies(interactions_df[feature], prefix = "u_" + feature))

    # merge all features into one data frame with user_id column
    users_df = reduce(lambda left, right: pd.merge(left, right, how = 'left', left_index = True, right_index = True), extracted_features)

    # insert missing column after one-hot due to not appearing [500-900] value in room_segment bucket in interactions_df
    users_df.insert(18, 'u_room_segment_[500-900]', 0)

    # insert missing user id column
    users_df.insert(0, 'user_id', user_ids)

    # in case of any NaN 
    users_df = users_df.fillna(0)

    # extract user features names
    user_features = list(users_df.columns.values[1:])

    return users_df, user_features

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['u_term_Christmas', 'u_term_Easter', 'u_term_HighSeason', 'u_term_LowSeason', 'u_term_MayLongWeekend', 'u_term_NewYear', 'u_term_OffSeason', 'u_term_WinterVacation', 'u_length_of_stay_bucket_[0-1]', 'u_length_of_stay_bucket_[2-3]', 'u_length_of_stay_bucket_[4-7]', 'u_length_of_stay_bucket_[8-inf]', 'u_rate_plan_Nonref', 'u_rate_plan_Standard', 'u_room_segment_[0-160]', 'u_room_segment_[160-260]', 'u_room_segment_[260-360]', 'u_room_segment_[360-500]', 'u_room_segment_[500-900]', 'u_n_people_bucket_[1-1]', 'u_n_people_bucket_[2-2]', 'u_n_people_bucket_[3-4]', 'u_n_people_bucket_[5-inf]', 'u_weekend_stay_False', 'u_weekend_stay_True']


,user_id,u_term_Christmas,u_term_Easter,u_term_HighSeason,u_term_LowSeason,u_term_MayLongWeekend,u_term_NewYear,u_term_OffSeason,u_term_WinterVacation,u_length_of_stay_bucket_[0-1],...,u_room_segment_[160-260],u_room_segment_[260-360],u_room_segment_[360-500],u_room_segment_[500-900],u_n_people_bucket_[1-1],u_n_people_bucket_[2-2],u_n_people_bucket_[3-4],u_n_people_bucket_[5-inf],u_weekend_stay_False,u_weekend_stay_True
0,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1
40,50,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,1
321,706,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,1
547,115,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,1
1270,1736,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,1,0
2785,96,0,0,1,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,1,0
11527,7779,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [173]:
def prepare_items_df(interactions_df):

    # clear data, leave only rows with unique item_id
    interactions_df = interactions_df.drop_duplicates(subset = ['item_id']).reset_index()

    # extract features as one-hot representation | base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']
    extracted_features = []

    # create data frames of features
    for feature in base_item_features:
        extracted_features.append(pd.get_dummies(interactions_df[feature], prefix = feature))

    # merge all features into one data frame with item_id column
    items_df = reduce(lambda left, right: pd.merge(left, right, how = 'left', left_index = True, right_index = True), extracted_features)
    items_df.insert(0, 'item_id', interactions_df['item_id'])

    # in case of any NaN
    items_df = items_df.fillna(0)

    # extract item_features names
    item_features = list(items_df.columns.values[1:])

    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['term_WinterVacation', 'term_Easter', 'term_OffSeason', 'term_HighSeason', 'term_LowSeason', 'term_MayLongWeekend', 'term_NewYear', 'term_Christmas', 'length_of_stay_bucket_[0-1]', 'length_of_stay_bucket_[2-3]', 'length_of_stay_bucket_[4-7]', 'length_of_stay_bucket_[8-inf]', 'rate_plan_Standard', 'rate_plan_Nonref', 'room_segment_[0-160]', 'room_segment_[160-260]', 'room_segment_[260-360]', 'room_segment_[360-500]', 'room_segment_[500-900]', 'n_people_bucket_[1-1]', 'n_people_bucket_[2-2]', 'n_people_bucket_[3-4]', 'n_people_bucket_[5-inf]', 'weekend_stay_True', 'weekend_stay_False']


,item_id,term_WinterVacation,term_Easter,term_OffSeason,term_HighSeason,term_LowSeason,term_MayLongWeekend,term_NewYear,term_Christmas,length_of_stay_bucket_[0-1],...,room_segment_[160-260],room_segment_[260-360],room_segment_[360-500],room_segment_[500-900],n_people_bucket_[1-1],n_people_bucket_[2-2],n_people_bucket_[3-4],n_people_bucket_[5-inf],weekend_stay_True,weekend_stay_False
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
1,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
2,2,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,1
3,3,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
5,5,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
6,6,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [174]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
    
        positive_interactions_dict = {key: 1 for key in zip(interactions_df['user_id'], interactions_df['item_id'])}
        #negative_interactions_dict = defaultdict(lambda: 0)
        
        negative_interactions = []
        negative_interactions_size = 0
        negative_interactions_goal = self.n_neg_per_pos * len(interactions_df)

        unique_user_ids = interactions_df['user_id'].unique()
        unique_item_ids = interactions_df['item_id'].unique()
        
        # generating negative interactions
        while negative_interactions_size < negative_interactions_goal:

            random_user = np.random.choice(unique_user_ids)
            random_item = np.random.choice(unique_item_ids)
            candidate_tuple = (random_user, random_item)

            # is it worth to generate only unique negative interactions ? ...
            if candidate_tuple not in positive_interactions_dict:# and candidate_tuple not in negative_interactions_dict:
                #negative_interactions_dict[candidate_tuple] = 0
                negative_tuple = (random_user, random_item, 0)
                negative_interactions.append(negative_tuple)
                negative_interactions_size += 1
        
        interactions_df = pd.concat([interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
    
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        # prepare users_df
        users_df = pd.merge(users_df, self.users_df, on = 'user_id', how = 'left')

        # prepare items_df
        items_df, item_features = prepare_items_df(items_df)

        # Score the items
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        # create profile of average user based on average values in user features in order to treat new user as an average user
        average_user = self.users_df.loc[:, self.user_features].mean()

        for ix, user in users_df.iterrows():
            
            # check if it's new user
            is_new = pd.isna(user.values).any()

            if is_new:
                # assign average features
                user_id = user['user_id']
                user = average_user.copy()
                user['user_id'] = user_id
            
            # generate cross product
            user_df = user.to_frame().transpose()
            product_df = pd.merge(items_df, user_df, how="cross")
            product = product_df.loc[:, user_features + item_features].values

            # score
            scores = self.model.predict(product)

            # get best results
            scores_df = pd.DataFrame({'item_id': items_df['item_id'], 'score': scores}).sort_values(by = ['score'], ascending = False)
            chosen_ids = scores_df['item_id'].head(n_recommendations).tolist()

            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores_df.loc[scores_df['item_id'] == item_id, 'score'].values[0]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [175]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [176]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [177]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,50,0.989271,OffSeason,[2-3],Nonref,[160-260],[3-4],True
1,1.0,54,0.986981,OffSeason,[2-3],Nonref,[160-260],[2-2],True
2,1.0,22,0.976778,OffSeason,[2-3],Standard,[160-260],[3-4],True
3,1.0,23,0.975808,OffSeason,[4-7],Standard,[160-260],[2-2],True
4,1.0,32,0.974719,OffSeason,[2-3],Standard,[160-260],[2-2],True
5,1.0,60,0.963375,OffSeason,[4-7],Nonref,[160-260],[2-2],True
6,1.0,14,0.962,OffSeason,[4-7],Standard,[160-260],[3-4],True
7,1.0,75,0.93629,OffSeason,[4-7],Nonref,[160-260],[3-4],True
8,1.0,56,0.907385,OffSeason,[2-3],Nonref,[160-260],[3-4],False
9,1.0,21,0.898438,OffSeason,[2-3],Standard,[160-260],[2-2],False


# Tuning method

In [178]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [179]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

### LinearRegressionCBUIRecommender params tuning

In [181]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df, param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [14:28<00:00, 86.85s/trial, best loss: -0.2280998730427423]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.240326,0.049219,0.094502,0.113016,0.134016


Best parameters:
{'n_neg_per_pos': 2.0}


### SVRCBUIRecommender params tuning

In [45]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df, param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

### RandomForestCBUIRecommender params tuning

In [188]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df, param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 100/100 [3:45:02<00:00, 135.03s/trial, best loss: -0.23360135421074904] 


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.049219,0.130007,0.17685,0.234895,0.049219,0.094502,0.114066,0.132694


Best parameters:
{'max_depth': 7.0, 'min_samples_split': 11.0, 'n_estimators': 248.0, 'n_neg_per_pos': 7.0}


### XGBoostCBUIRecommender params tuning

In [27]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df, param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 300/300 [6:04:49<00:00, 72.96s/trial, best loss: -0.23444773592890394]   


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.049898,0.130346,0.175832,0.244399,0.049898,0.095722,0.114746,0.136526


Best parameters:
{'learning_rate': 0.012713530448885008, 'max_depth': 6.0, 'min_samples_split': 15.0, 'n_estimators': 142.0, 'n_neg_per_pos': 4.0}


# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

### LinearRegressionCBUIRecommender

In [203]:
cb_user_item_recommender = LinearRegressionCBUIRecommender()

linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.233198,0.049219,0.094502,0.113016,0.131796


### LinearRegressionCBUIRecommenderTunned

In [211]:
best_params = {
    'n_neg_per_pos': 2.0
}

cb_user_item_recommender = LinearRegressionCBUIRecommender(**best_params)

linear_tunned_cbui_tts_results = [['LinearRegressionCBUIRecommenderTunned'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

linear_tunned_cbui_tts_results = pd.DataFrame(linear_tunned_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_tunned_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommenderTunned,0.049219,0.130007,0.174134,0.240326,0.049219,0.094502,0.113016,0.134097


### SVRCBUIRecommender

In [214]:
cb_user_item_recommender = SVRCBUIRecommender()

svr_cbui_tts_results = [['SVRCBUIRecommender'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

svr_cbui_tts_results = pd.DataFrame(svr_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(svr_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,SVRCBUIRecommender,0.043788,0.124915,0.174813,0.245757,0.043788,0.088796,0.109676,0.132503


### RandomForestCBUIRecommender

In [217]:
cb_user_item_recommender = RandomForestCBUIRecommender()

random_forest_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

random_forest_cbui_tts_results = pd.DataFrame(random_forest_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(random_forest_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.047862,0.104888,0.173456,0.239308,0.047862,0.082152,0.110284,0.131229


### RandomForestCBUIRecommenderTunned

In [219]:
best_params = {
    'max_depth': 7.0, 
    'min_samples_split': 7.0, 
    'n_estimators': 217.0, 
    'n_neg_per_pos': 4.0
}

cb_user_item_recommender = RandomForestCBUIRecommender(**best_params)

random_forest_tunned_cbui_tts_results = [['RandomForestCBUIRecommenderTunned'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

random_forest_tunned_cbui_tts_results = pd.DataFrame(random_forest_tunned_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(random_forest_tunned_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommenderTunned,0.049219,0.130007,0.177529,0.243041,0.049219,0.094502,0.114329,0.13511


### XGBoostCBUIRecommender

In [190]:
cb_user_item_recommender = XGBoostCBUIRecommender()

xg_boost_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

xg_boost_cbui_tts_results = pd.DataFrame(xg_boost_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(xg_boost_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.0,0.001358,0.002037,0.002716,0.0,0.000857,0.001134,0.001339


### XGBoostCBUIRecommenderTunned

In [197]:
best_params = {
    'learning_rate': 0.02674848891060033, 
    'max_depth': 6.0, 
    'min_samples_split': 8.0, 
    'n_estimators': 100.0, 
    'n_neg_per_pos': 6.0
}

cb_user_item_recommender = XGBoostCBUIRecommender(**best_params)

xg_boost_tunned_cbui_tts_results = [['XGBoostCBUIRecommenderTunned'] + list(evaluate_train_test_split_implicit(cb_user_item_recommender, interactions_df, items_df))]

xg_boost_tunned_cbui_tts_results = pd.DataFrame(xg_boost_tunned_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(xg_boost_tunned_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommenderTunned,0.049559,0.130007,0.174474,0.247794,0.049559,0.094672,0.113317,0.136555


### Amazon Recommender

In [199]:
amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.044128,0.118805,0.160896,0.223693,0.044128,0.086755,0.104347,0.124505


### Comparison

In [293]:
tts_results = pd.concat([linear_cbui_tts_results, 
                        linear_tunned_cbui_tts_results,
                        svr_cbui_tts_results,
                        random_forest_cbui_tts_results,
                        random_forest_tunned_cbui_tts_results,
                        xg_boost_cbui_tts_results,
                        xg_boost_tunned_cbui_tts_results,
                        amazon_tts_results]
                    ).reset_index(drop=True)
                    
tts_results = tts_results.sort_values(by = ['HR@10']).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.000000,0.001358,0.002037,0.002716,0.000000,0.000857,0.001134,0.001339
1,AmazonRecommender,0.044128,0.118805,0.160896,0.223693,0.044128,0.086755,0.104347,0.124505
2,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.233198,0.049219,0.094502,0.113016,0.131796
3,RandomForestCBUIRecommender,0.047862,0.104888,0.173456,0.239308,0.047862,0.082152,0.110284,0.131229
4,LinearRegressionCBUIRecommenderTunned,0.049219,0.130007,0.174134,0.240326,0.049219,0.094502,0.113016,0.134097
5,RandomForestCBUIRecommenderTunned,0.049219,0.130007,0.177529,0.243041,0.049219,0.094502,0.114329,0.135110
6,SVRCBUIRecommender,0.043788,0.124915,0.174813,0.245757,0.043788,0.088796,0.109676,0.132503
7,XGBoostCBUIRecommenderTunned,0.049559,0.130007,0.174474,0.247794,0.049559,0.094672,0.113317,0.136555


### Conclusion

Unfortunately, it was not possible to achieve HR 0.25 despite strenuous attempts, but the tuning paid off, you can see that the results after tuning are much better, especially on the example of XGBoost model. However, almost every recommender beat amazon, XGBoost without tuning had no chance due to the large number of parameters.